In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-20 21:43:29--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-08-20 21:43:29 (29.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open("input.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [ ]:
print(f"length of dataset in characters: {len(text)}")

length of dataset in characters: 1115394


In [ ]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
chars = sorted(list(set(text)))
vocab_sz = "".join(chars)
print(vocab_sz)
print(len(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
#creating a simple tokenizer
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i: ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda c: [itos[i] for i in c]

print(encode("hii there"))
print(''.join(decode(encode("hii there"))))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
#encoding entirety of input text with tokenizer
import torch
data = torch.tensor(encode(text), dtype= torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
#90/10 data split
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

print(train[:10])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [ ]:
#visualization of the autoregressive techniuque used by transformers for language modeling
block_sz = 8
x = train[:block_sz]
y = train[1:block_sz + 1]
for t in range(block_sz):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")


when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
torch.manual_seed(80)
batch_sz = 4 #number of input sequences to be processed in parallel
block_sz = 8 #maximum context length of predictions

def get_batch(split):
  #generate a small batch of data of inputs x and targets y
  data = train if split == 'train' else val
  ix = torch.randint(len(data) - block_sz, (batch_sz,))
  x = torch.stack([data[i:i+block_sz] for i in ix])
  y = torch.stack([data[i+1:i+block_sz+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_sz): #batch dimension
  for t in range(block_sz): #time dimension
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when the input is {context.tolist()}, the output is {target}")

inputs:
torch.Size([4, 8])
tensor([[52, 42,  1, 42, 47, 42,  1, 58],
        [39, 42, 50, 63,  1, 57, 54, 53],
        [51,  1, 58, 53,  1, 39, 50, 50],
        [63, 53, 59,  6,  1, 59, 52, 41]])
targets:
torch.Size([4, 8])
tensor([[42,  1, 42, 47, 42,  1, 58, 56],
        [42, 50, 63,  1, 57, 54, 53, 49],
        [ 1, 58, 53,  1, 39, 50, 50, 11],
        [53, 59,  6,  1, 59, 52, 41, 50]])
-----
when the input is [52], the output is 42
when the input is [52, 42], the output is 1
when the input is [52, 42, 1], the output is 42
when the input is [52, 42, 1, 42], the output is 47
when the input is [52, 42, 1, 42, 47], the output is 42
when the input is [52, 42, 1, 42, 47, 42], the output is 1
when the input is [52, 42, 1, 42, 47, 42, 1], the output is 58
when the input is [52, 42, 1, 42, 47, 42, 1, 58], the output is 56
when the input is [39], the output is 42
when the input is [39, 42], the output is 50
when the input is [39, 42, 50], the output is 63
when the input is [39, 42, 50, 63], 